In [11]:
import os 
os.chdir("/Users/work/Documents/GitHub/Bus-Weather-Impacts")
from src.utils import *
import pandas as pd
import os
import osmnx as ox
import geopandas as gpd
import networkx as nx
from sklearn.neighbors import KDTree
import numpy as np

In [35]:
buses = read_parquet_from_tar_gz("https://urbantech-public.s3.amazonaws.com/DO-NOT-DELETE-BUSOBSERVATORY-PUBLIC-DATASET/one-system-day.tar.gz")

In [18]:
df = pd.read_parquet('https://urbantech-public.s3.amazonaws.com/pickup/dump_NYC_GTFSRT/20240108_183520_00129_ygreg_ffbd96b2-8380-4c00-adbc-536f2fcb8079')

In [21]:
df['vehicle.trip.start_date'].value_counts()

vehicle.trip.start_date
20231024    137765
20230727    131072
20231214    126179
20231113    125261
20231201    122015
             ...  
20230604     26970
20231029     24638
20230529     20535
20230611     20336
20230427     16044
Name: count, Length: 257, dtype: int64

In [4]:
place = "New York City, New York, USA"
G = ox.graph_from_place(place, network_type='drive')

# Convert graph nodes to a DataFrame for KDTree
nodes = ox.graph_to_gdfs(G, edges=False).reset_index()
tree = KDTree(nodes[['y', 'x']], metric='euclidean')

In [ ]:
# Query the KDTree for nearest node to each point
nearest_nodes = tree.query(np.array(buses[['lat', 'lon']]), k=1, return_distance=False)
buses['nearest_node'] = nearest_nodes.flatten()

# Map node IDs to OSM IDs
buses['nearest_osm_id'] = buses['nearest_node'].map(nodes['osmid'])
buses = buses.merge(nodes, left_on = "nearest_osm_id", right_on = "osmid")
buses = gpd.GeoDataFrame(buses, geometry='geometry')

In [ ]:
buses["time_diff"] = buses.groupby("trip_id")["timestamp"].diff().dt.total_seconds()
buses = buses.query("time_diff >= 60")

In [ ]:
buses_with_nodes = buses.sort_values(["trip_id", "timestamp"]).drop_duplicates(subset = ["trip_id", "nearest_osm_id"],  keep = "first").to_crs(2263)
buses_with_nodes = buses_with_nodes[["route", "timestamp", "trip_id", "osmid", "vehicle_id",  "geometry"]]
buses_with_nodes["prev_osmid"] = buses_with_nodes.groupby("trip_id")["osmid"].shift()

In [ ]:
def compute_distance(row, graph):
    try:
        return nx.shortest_path_length(graph, row['prev_osmid'], row['osmid'], weight='length')
    except nx.NetworkXNoPath:
        return float('nan')
    
def compute_euclid_dists(node_pairs, nodes_points):
    node_pairs = node_pairs.merge(nodes_points, left_on = "osmid", right_on = "osmid", how = "left").merge(nodes_points, right_on = "osmid", left_on = "prev_osmid", how = "left", suffixes = ["curr", "prev"])
    node_pairs["x_diff_sq"] = (node_pairs["xcurr"] - node_pairs["xprev"])**2
    node_pairs["y_diff_sq"] = (node_pairs["ycurr"] - node_pairs["yprev"])**2

    return (node_pairs["x_diff_sq"] + node_pairs["y_diff_sq"]) ** (1/2)

nodes_points = nodes.to_crs(2263)
nodes_points["x"] = nodes_points.geometry.x
nodes_points["y"] = nodes_points.geometry.y
nodes_points = nodes_points[["osmid", "x", "y"]]

unqiue_node_pairs = buses_with_nodes[["osmid", "prev_osmid"]].dropna().drop_duplicates().astype("Int64").reset_index()
unqiue_node_pairs["distance_meters_osm"] = unqiue_node_pairs.apply(compute_distance, graph=G, axis=1)
unqiue_node_pairs["distance_meters_euclid"] = compute_euclid_dists(unqiue_node_pairs, nodes_points) / 3.28

unqiue_node_pairs.to_parquet("data/node_pairs.parquet")

unique_node_pairs = pd.read_parquet("data/node_pairs.parquet")
buses_with_nodes = buses_with_nodes.merge(unique_node_pairs, how = "left")
buses_with_nodes["osmid"] = buses_with_nodes["osmid"].astype(str)
buses_with_nodes["prev_osmid"] = buses_with_nodes["prev_osmid"].astype("Int64").astype(str)

ValueError: invalid literal for int() with base 10: '<NA>'

In [31]:
buses_with_nodes["time_diff_seconds"] = buses_with_nodes.groupby("trip_id")["timestamp"].diff().dt.total_seconds()
buses_with_nodes["speed_mph_osm"] =  (buses_with_nodes["distance_meters_osm"] / 1609) / (buses_with_nodes["time_diff_seconds"] / 3600)
buses_with_nodes["speed_mph_euclid"] =  (buses_with_nodes["distance_meters_euclid"] / 1609) / (buses_with_nodes["time_diff_seconds"] / 3600)

In [33]:
unqiue_node_pairs

,osmid,prev_osmid,distance_meters_osm,distance_meters_euclid
754333,42971112,42971120,410.265,NaN
1567424,42962132,42971112,570.504,NaN
360648,410544431,42965719,626.869,NaN
118352,42947856,42944036,699.414,281.108829
1801165,42951005,42947856,445.257,NaN
...,...,...,...,...
1323207,42440323,42445867,248.212,NaN
1130478,447760277,42986987,976.978,NaN
1866765,8264070984,42978105,797.705,NaN
83261,410544474,42964909,3852.038,483.940807


In [20]:
compute_euclid_dists(unqiue_node_pairs, nodes_points) / 3.28

0          409.824293
1          401.598082
2          347.602321
3          517.578622
4          446.421753
             ...     
210493     184.244547
210494     942.196053
210495     788.538731
210496    3570.562742
210497    4228.918867
Length: 210498, dtype: float64

In [ ]:
nodes.query("osmid == 42949762 | osmid == 598367717")

In [ ]:
buses_with_nodes.query("speed_mph_osm > 100")

In [ ]:
buses_with_nodes['distance_diff_feet'] = buses_with_nodes.groupby('trip_id').apply(calculate_distances).reset_index(level=0, drop=True)

In [ ]:
buses.query("route == 'M96'").plot()

In [ ]:
data.head(50)

In [ ]:
plot_osm_path(start, end)

In [ ]:
path_nodes = nx.shortest_path(G, 598367717, 42949762)


In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
nodes.query("osmid.isin(@path_nodes)").to_crs(3857).plot(ax = ax)

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

plt.show()


In [ ]:
data_url = "https://ndownloader.figshare.com/files/20232174"
db = gpd.read_file(data_url)
ax = db.plot(color="red", figsize=(9, 9))
ctx.add_basemap(ax, crs=db.crs.to_string())
